# CoSim - RTED

In [1]:
import numpy as np

import andes
import ams

import pandas as pd

import json

import cvxpy as cp

In [2]:
ams.__version__

'0.6.4.post103.dev0+gf8d04e9'

In [3]:
andes.config_logger(stream_level=20)

In [4]:
sa = andes.load(andes.get_case('kundur/kundur_coi.xlsx'),
                setup=True,
                no_output=True,
                )


Working directory: "/home/jwang/work/ams/dev/demo"
> Loaded config from file "/home/jwang/.andes/andes.rc"
> Loaded generated Python code in "/home/jwang/.andes/pycode".
Parsing input file "/home/jwang/.local/lib/python3.10/site-packages/andes/cases/kundur/kundur_coi.xlsx"...
Input file parsed in 0.1589 seconds.
System internal structure set up in 0.0449 seconds.


In [5]:
ams.config_logger(stream_level=10)

In [6]:
sp = ams.load(ams.get_case('ieee14/ieee14_rted.xlsx'),
              setup=True,
              )

Working directory: "/home/jwang/work/ams/dev/demo"
Input format guessed as xlsx.
Parsing input file "/home/jwang/work/ams/ams/cases/ieee14/ieee14_rted.xlsx"...
Input file parsed in 0.0636 seconds.
Adjust bus index to start from 0.
System set up in 0.0098 seconds.


In [7]:
sp.RTED.run()

Setup model of RTED
RTED pg0 are all zeros, correct to p0.
RTED data check passed.
- Generating symbols for RTED
Set constrs pb: self.constrs["pb"]=cp.sum(self.routine.pd.v) - cp.sum(self.pg) == 0
Set constrs lub: self.constrs["lub"]=self.routine.PTDF1.v @ (self.pg - self.routine.pd1.v) - self.routine.PTDF2.v @ self.routine.pd2.v - self.routine.rate_a.v <= 0
Set constrs llb: self.constrs["llb"]=- self.routine.PTDF1.v @ (self.pg - self.routine.pd1.v) + self.routine.PTDF2.v @ self.routine.pd2.v - self.routine.rate_a.v <= 0
Set constrs rbu: self.constrs["rbu"]=self.routine.prs.v @ self.pru - self.routine.du.v == 0
Set constrs rbd: self.constrs["rbd"]=self.routine.prs.v @ self.prd - self.routine.dd.v == 0
Set constrs rru: self.constrs["rru"]=self.pg + self.pru - self.routine.pmax.v <= 0
Set constrs rrd: self.constrs["rrd"]=-self.pg + self.prd - self.routine.pmin.v <= 0
Set constrs rgu: self.constrs["rgu"]=self.pg - self.routine.pg0.v - self.routine.R10.v <= 0
Set constrs rgd: self.constrs[

In [8]:
sp.RTED.smooth()

Setup model for ACOPF
ACOPF data check passed.
- Generating symbols for ACOPF
Set constrs pb: self.constrs["pb"]=cp.sum(self.routine.pd.v) - cp.sum(self.pg) == 0
ACOPF model set up in 0.0054 seconds.
Adjust bus index to start from 0.
PYPOWER Version 5.1.4, 27-June-2018
 -- AC Optimal Power Flow



Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011


ACOPF completed in 0.6783 seconds with exit code 0.
RTED is smoothed with ACOPF.


Converged!


True

In [9]:
sa = sp.to_andes(setup=False,
                 addfile=andes.get_case('ieee14/ieee14_wt3.xlsx'),
                 overwrite=True,
                 keep=False,
                 no_output=True,
                 )

sa.add("Toggle", dict(model='SynGen', dev="GENROU_4", t=11.0))

sa.setup()

JSON file written to "ieee14_rted.json"
System convert to ANDES in 0.1631 seconds, save to "ieee14_rted.json".
Working directory: "/home/jwang/work/ams/dev/demo"
> Loaded config from file "/home/jwang/.andes/andes.rc"
> Reloaded generated Python code of module "pycode".
Parsing input file "ieee14_rted.json"...
Input file parsed in 0.0033 seconds.
Converted file is removed. Set "keep = True" to keep it.
Addfile format guessed as xlsx.
Parsing additional file "/home/jwang/.local/lib/python3.10/site-packages/andes/cases/ieee14/ieee14_wt3.xlsx"...
Power flow models exist in the addfile. Only dynamic models will be kept.
Correct SynGen idx to match StaticGen.
Correct RenGen idx to match StaticGen.
Addfile parsed in 0.1274 seconds.
AMS system 0x7f4d86214430 link to ANDES system 0x7f4d845f5cf0 in 0.1350 seconds.
System internal structure set up in 0.0485 seconds.


True

In [10]:
sp.dyn.send()

RTED results has been smoothed.
Sending RTED results to ANDES...
Sending results to <pflow> models...
Var <RTED.vBus> is mapped to <v0> of Bus.


True

In [11]:
sa.PFlow.run()

-> System connectivity check results:
  No islanded bus detected.
  System is interconnected.
  Each island has a slack bus correctly defined and enabled.

-> Power flow calculation
           Numba: Off
   Sparse solver: KLU
 Solution method: NR method
Power flow initialized in 0.0087 seconds.
0: |F(x)| = 0.5605182162
1: |F(x)| = 0.006202200463
2: |F(x)| = 5.819382991e-06
3: |F(x)| = 6.960640397e-12
Converged in 4 iterations in 0.0132 seconds.


True

In [12]:
sp.RTED.pg0

RParam: StaticGen.pg0, v=[0. 0. 0. 0. 0.]

In [13]:
idx_tg = sa.TGOV1.idx.v
vmax0 = sa.TGOV1.get(src='VMAX', attr='v', idx=idx_tg)
sa.TGOV1.set(src='VMAX', attr='v', idx=idx_tg, value=10 * vmax0)
vmin0 = sa.TGOV1.get(src='VMIN', attr='v', idx=idx_tg)
sa.TGOV1.set(src='VMIN', attr='v', idx=idx_tg, value=np.zeros_like(vmin0))

True

In [14]:
sa.TDS.config.tf = 10
sa.TDS.run()


-> Time Domain Simulation Summary:
Sparse Solver: KLU
Simulation time: 0.0-10 s.
Fixed step size: h=33.33 ms. Shrink if not converged.
Initialization for dynamics completed in 0.1246 seconds.
Initialization was successful.


  0%|          | 0/100 [00:00<?, ?%/s]

<Toggle Toggler_1>: Line.Line_1 status changed to 0 at t=1.0 sec.
<Toggle Toggler_2>: Line.Line_1 status changed to 1 at t=1.1 sec.


Simulation to t=10.00 sec completed in 2.2809 seconds.


True

In [15]:
# sa.TDS.config.tf = 20
# sa.TDS.run()

In [16]:
sp.dyn.sa

In [17]:
sp.dyn.send()

RTED results has been smoothed.
Sending RTED results to ANDES...
Sending results to <tds> models...
Generator online status are sent to dynamic generators.
Dynamic has been initialized, Bus angle and voltage are skipped.
Var <RTED.pg> is mapped to <p0> of StaticGen.


In [18]:
sp.dyn.receive()

Receiving ANDES results to RTED.
Receiving <tds> results to RTED...
Generator online status are received from dynamic generators.
Generator output power are received from dynamic generators.


True